<h2>Import packages and data</h2>

In [1]:
from scipy.io import netcdf
from scipy.stats import entropy
import similarity_measures as sim
import calculations as calc
import comparing as comp
import combining as comb
import plots
import numpy as np
import matplotlib as plt

In [10]:
import ipywidgets as widgets
from ipywidgets import interact, fixed

In [4]:
plt.rcParams.update({'font.size': 25})

In [2]:
nc_l30 = netcdf.netcdf_file("data/era-int_pl_1979-2019-mm-l30-u.nc")

<h2>Derive QBO</h2>

In [3]:
lon = calc.convert_coordinates_to_grid(nc_l30.variables["longitude"][:], 180)
lat = calc.convert_coordinates_to_grid(nc_l30.variables["latitude"][:], 0)

lon_step = 256 #Around the whole equator
lat_step = calc.convert_coordinates_to_grid(nc_l30.variables["latitude"][:], 0) - calc.convert_coordinates_to_grid(nc_l30.variables["latitude"][:], 5)


u_l30 = nc_l30.variables["u"]

#Deseasonalize
u_l30 = calc.deseasonalize_map(u_l30)

qbo_equator = calc.derive(u_l30[:, :, :, :], lat, lon, 0, lat_step, lon_step)

<h1>Widget version</h1>

<h3>Preparation</h3>

Define similarity measures to use

In [5]:
measures = [sim.pearson_correlation_abs, sim.mutual_information, comp.invert(sim.manhattan_distance)]

In [6]:
measure_labels = ["Absolute Pearson's Correlation", "Mutual Information", "Manhattan Distance (Inverted)"]

Compute the similarities and the summary maps

In [7]:
#Calculation
similarities = []
mean_map = np.zeros(u_l30[0, 0, :, :].shape)
agreement = np.zeros_like(mean_map)

for measure in measures:
    similarity = calc.calculate_series_similarity(u_l30, qbo_equator, 0, measure)
    if (measure != sim.pearson_correlation or measure !=sim.pearson_correlation_abs):
        similarity = comp.binning_values_to_quantiles(similarity)
    similarities.append(similarity)

agreement = np.std(similarities, axis=0)
mean_map = np.mean(similarities, axis=0)

Define a function that plots the remaining points, based on the selected thresholds

In [8]:
def plot_agreement(mean_map, agreement_map, value_threshold, agreement_threshold, values_high, agree_high):
    map = np.ones_like(mean_map)

    agreement_filtered = calc.filter_map(agreement, agreement_threshold, high=agree_high)
    mean_map_filtered = calc.filter_map(mean_map, value_threshold, high=values_high)

    map = map * agreement_filtered * mean_map_filtered
    
    fig, axis = plt.pyplot.subplots(1, 1, figsize=(30, 20))
    fig.suptitle("Areas where {} agree there is a dependency".format(measure_labels))
    
    plots.plot_map(map, axis, colorbar=False)

<h3>Create an interactive widget to set the parameters</h3>

In [11]:
value_slider = widgets.FloatSlider(value=0.8, min=0, max=1, step=0.05, description='Value Threshold:')
agreement_slider = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='Agreement Threshold:')
fil_val_high = widgets.RadioButtons(
    options=[('High dependencies', True), ('Low dependencies', False)],
    value=True,
    description='Values:',
)
fil_agree_high = widgets.RadioButtons(
    options=[('Agree', False), ('Disagree', True)],
    value=False,
    description='Agreement',
)
widgets.interact(plot_agreement, value_threshold=value_slider, agreement_threshold=agreement_slider, values_high=fil_val_high, agree_high=fil_agree_high, mean_map=fixed(mean_map), agreement_map=fixed(agreement)) 

interactive(children=(FloatSlider(value=0.8, description='Value Threshold:', max=1.0, step=0.05), FloatSlider(…

<function __main__.plot_agreement(mean_map, agreement_map, value_threshold, agreement_threshold, values_high, agree_high)>